In [1]:
import time, threading

def long_io(cb):
    def fun(callback):
        print('long io start')
        time.sleep(5)
        print('finish long io')
        result = "long io"
        callback(result)
    t = threading.Thread(target=fun, args=(cb,))
    t.start()
    
def on_finish(res):
    print(res)

def a():
    print('start a')
    res = long_io(on_finish)
    print('leave a')
    
def b():
    print('start b')
    print('finish b')
    
def main():
    a()
    b()

if __name__ == '__main__':
    main()

start a
long io startleave a
start b
finish b

finish long io
long io


In [ ]:
import time, threading

def gen_coroutine(f):
    def wrapper(*args, **kwargs):
        gen_f = f()                     # gen_f 为生成器 req_a
        r = gen_f.next()                # r 为生成器 long_io
        def fun(g):
            res = g.next()              # 执行生成器 long_io
            try:
                gen_f.send(res)         # 将结果返回 req_a 使其继续执行
            except StopIteration:
                pass
        t = threading.Thread(target=fun, args=(r,))
        t.start()
    return wrapper

def long_io():
    print('long_io start')
    time.sleep(5)
    print('完成IO操作，yield回操作结果')
    yield 'io result'

@gen_coroutine
def req_a():
    print('start a')
    res = yield long_io()
    print('res: %s' % res)
    print('finish a')

def req_b():
    print('start b')
    time.sleep(2)
    print('finish b')

def main():
    req_a()
    req_b()

if __name__ == '__main__':
    main()